In [15]:
import pandas as pd
import json
import re

In [16]:
data = pd.read_json('test.json')
data = pd.DataFrame(data)

testData = data.loc[0, 'text']
testData = str(testData)

In [14]:
p=re.compile(r'\b[Aa]ls\b')
#p.findall(testData)
#len(p.findall(testData))

In [13]:
# finds locations (start, end) of all matches
p=re.compile(r'als')
iterator = p.finditer(testData)

positions = []

for match in iterator:
    newPosition = match.start()
    positions.append(newPosition)

#positions

In [4]:
output = data
output['matches_schop'] = ''
output['pos_schop'] = ''
output['frac_schop'] = ''

for i, row in output.iterrows():
    chunk = row['text']
    chunk = str(chunk)

    pAll=re.compile(r'.*')
    iterAll = pAll.finditer(chunk)
    for match in iterAll:
        span = match.start()
    output.at[i, 'length'] = span

    pSchop = re.compile(r'\b[Ss]chopenhauer\w+') # will match all words beginning with 'Schopenhauer' or 'schopenhauer'
    output.at[i, 'matches_schop'] = pSchop.findall(chunk)
    output.at[i, 'num_schop'] = len(pSchop.findall(chunk))
    
    positions = []  
    fractions = []
    iterSchop = pSchop.finditer(chunk)
    for match in iterSchop:
        newPosition = match.start()
        newFraction = newPosition/span
        positions.append(newPosition)
        fractions.append(newFraction)

    if (positions == []):
        output.at[i, 'pos_schop'] = ''
    else:
        output.at[i, 'pos_schop'] = positions

    if(fractions == []):
        output.at[i, 'frac_schop'] = ''
    else: 
        output.at[i, 'frac_schop'] = fractions

output

,year,group,text,matches_schop,pos_schop,frac_schop,length,num_schop
0,1869,1,[Wer heutzutage von Aeschylus Sophocles Euripi...,[],,,50467.0,0.0
1,1869,2,"[Das Christenthum, ursprünglich Sache des Tale...",[],,,17195.0,0.0
2,1869,3,[Wir sind leider gewöhnt die Künste in der Ver...,[Schopenhauersche],[16071],[0.42369038517307744],37931.0,1.0
3,1870,4,"[Erlangen Samstag den 20. August., Hier seit a...",[],,,6619.0,0.0
4,1870,5,[Gedanken zu\n„die Tragoedie und die\nFreigeis...,[schopenhauerischen],[21781],[0.4785034820624355],45519.0,1.0
...,...,...,...,...,...,...,...,...
171,1888,21,"[Teichm<üller>, Scept<iques> gr<ecs>, Spir, A<...",[],,,4258.0,0.0
172,1888,22,[Randbemerkung zu einer niaserie anglaise. — „...,[],,,18594.0,0.0
173,1888,23,[Auch ein Gebot der Menschenliebe. — Es giebt ...,[],,,23305.0,0.0
174,1888,24,[Ecce homo\nOder:\nwarum ich Einiges mehr weis...,"[Schopenhauern, Schopenhauers]","[2394, 25747]","[0.06680992381324477, 0.7185276142103647]",35833.0,2.0


In [31]:
#reformatting so that svelte app can more easily work with data

## MUST MANUALLY DELETE data_jsonified.json AND PUT IN AN EMPTY []
array = data
array['matches_schop'] = ''
array['pos_schop'] = ''
array['frac_schop'] = ''
rowNum = 0

indexer = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]*9

def rowFinder(i):
    if i <= 19:
        rowNum = 0
    elif i > 19 and i <= 38:
        rowNum = 1
    elif i > 38 and i <= 57:
        rowNum = 2
    elif i > 57 and i <= 76:
        rowNum = 3
    elif i > 76 and i <= 95:
        rowNum = 4
    elif i > 95 and i <= 114:
        rowNum = 5
    elif i > 114 and i <= 133:
        rowNum = 6
    elif i > 133 and i <= 152:
        rowNum = 7
    elif i > 152 and i <= 171:
        rowNum = 8
    
    return rowNum


for i, row in array.iterrows():
    chunk = row['text']
    chunk = str(chunk)

    pAll=re.compile(r'.*')
    iterAll = pAll.finditer(chunk)
    for match in iterAll:
        span = match.start()
    text_length = span

    pSchop = re.compile(r'\b[Ss]chopenhauer\w+') # will match all words beginning with 'Schopenhauer' or 'schopenhauer'
    matches_schop = pSchop.findall(chunk)
    num_matches = len(pSchop.findall(chunk))
    
    positions = []  
    fractions = []
    iterSchop = pSchop.finditer(chunk)
    for match in iterSchop:
        newPosition = match.start()
        newFraction = newPosition/span
        positions.append(newPosition)
        fractions.append(newFraction)
    
    j = {
                'index': indexer[i],
                'row': rowFinder(i),
                'year': int(row['year']),
                'group': int(row['group']),
                'text': row['text'],
                'total_length': text_length,
                'schop_matches': {
                    'num_matches': num_matches,
                    'matched_text': matches_schop,
                    'matched_positions': positions,
                    'matched_fractions': fractions,
        }
    }

    with open('data_jsonified.json', 'r') as file:
        a = json.load(file)
        a.append(j)
    with open('data_jsonified.json', 'w') as file:
        json.dump(a, file, indent=4)

    j = {}

UnboundLocalError: local variable 'rowNum' referenced before assignment

In [ ]:
    # if num_matches = 0:

    # else:
    #     for i in num_matches:
    #         j = {
    #             'index': i,
    #             'year': int(row['year']),
    #             'group': int(row['group']),
    #             'text': row['text'],
    #             'length': text_length,
    #             'schop_matches': {
    #                 'num_matches': num_matches,
    #                 'matched_text': matches_schop,
    #                 'matched_positions': positions,
    #                 'matched_fractions': fractions,
    #                 'length': text_length
    #             }
    # }